<h1> Scraper for Shopping Malls in SG </h1>

<h3> Scraping Malls from Wikipedia </h3>
<p> This is a simple webscraper to scrape the list of shopping malls off wikipedia.</p>

In [275]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore"
response = requests.get(url, timeout = 5)
content = BeautifulSoup(response.content, "html.parser")

In [244]:
# Parse HTML
soup = BeautifulSoup(response.content, "html.parser")

# Find the 2nd <tbody> tag that contains all malls
tbody_tag = soup.find_all('tbody')[1]

# Check within <tbody> tag
if tbody_tag:
    
    # Iterate over <th> tags within <tbody>
    for th_tag in tbody_tag.find_all('th'):
        # Remove entire <tr> if "Demolished" or "Under construction" is in the string for <th> text
        if any(keyword in th_tag.get_text() for keyword in ['Demolished', 'Under construction']):
            # Extract the parent <tr> tag and remove it
            tr_tag = th_tag.find_parent('tr')
            tr_tag.extract()
            
    # Remove <th> and <tr> lines now
    for th_tag in tbody_tag.find_all('th'):
        # Remove the th_tag
        th_tag.extract()

# Extract all relevant malls
mall_list_html = tbody_tag.find_all("ul")

# Turn it into a mall beautiful soup and extract into list
mall_soup = BeautifulSoup(''.join(str(tag) for tag in list(mall_list_html)))
mall_elements = mall_soup.find_all('li')

In [254]:
# Extract mall name
malls = []

for node in mall_elements:
    mallName = "".join(node.findAll(string = True))
    if len(mallName) > 1:
        malls.append("".join(node.findAll(string = True)))
malls = set(malls)

# Set upper case on all malls
malls = [element.upper() for element in malls]
malls

['HARBOURFRONT CENTRE',
 'NORTHPOINT CITY',
 'SUNTEC CITY MALL',
 'CATHAY CINELEISURE ORCHARD',
 'HEARTLAND MALL',
 '100 AM',
 'YEW TEE SQUARE',
 'SUNSHINE PLACE',
 'YEW TEE POINT',
 'THE CATHAY',
 'SIM LIM SQUARE',
 'CHANGI CITY POINT',
 '313@SOMERSET',
 'JUNCTION 10',
 'ION ORCHARD',
 'CITY SQUARE MALL',
 'THE RAIL MALL',
 'EASTPOINT MALL',
 'MARINA BAY LINK MALL',
 'MARINA SQUARE',
 '888 PLAZA',
 'BUGIS+',
 'COMPASS ONE',
 'GEK POH SHOPPING CENTRE',
 'TIONG BAHRU PLAZA',
 'ESPLANADE MALL',
 'SINGPOST CENTRE',
 'PARKWAY PARADE',
 'MYVILLAGE @ SERANGOON',
 'DJITSUN MALL BEDOK',
 'LUCKY PLAZA',
 'TAMPINES MART',
 'THE PARAGON',
 'THE WOODLEIGH MALL',
 'TAMPINES MALL',
 'DAWSON PLACE',
 'PLQ MALL',
 'WOODLANDS NORTH PLAZA',
 'CLAYMORE CONNECT',
 'SQUARE 2',
 'RIVERVALE PLAZA',
 'CLEMENTI MALL',
 'JEWEL CHANGI AIRPORT',
 'HILLION MALL',
 'PLAZA SINGAPURA',
 'WESTGATE',
 'LEISURE PARK KALLANG',
 'BEDOK MALL',
 'JURONG POINT',
 'CENTURY SQUARE',
 'FU LU SHOU COMPLEX',
 'ANCHORPOINT',
 'QUE

<h3> Use OneMap API for Mall Coordinates </h3>

In [255]:
# Function to get latitude and longitude from onemap api
def get_coordinates(location):
    searchQuery = f"https://developers.onemap.sg/commonapi/search?searchVal={location}&returnGeom=Y&getAddrDetails=Y"
    response = requests.get(searchQuery)
    
    try:
        result = response.json()['results'][0]
        latitude = result['LATITUDE']
        longitude = result['LONGITUDE']
        return [latitude, longitude]
    except: 
        return 'INVALID LOCATION'

In [256]:
# Function to search latitude and longitude for all names in a list
def searchOneMap(locationList):
    lat_long_locations = []
    invalid_locations = []
    for location in locationList:
        lat_long = get_coordinates(location)
        if lat_long != 'INVALID LOCATION': 
            location_details = {}
            location_details['name'] = location
            location_details['latitude'] = lat_long[0]
            location_details['longitude'] = lat_long[1]
            lat_long_locations.append(location_details)
        else:
            invalid_locations.append(location)
    return [lat_long_locations, invalid_locations]

In [257]:
# Retrieve lat long of malls
result = searchOneMap(malls)
mall_lat_long = result[0]
missing_malls = result[1]

In [258]:
# View missing malls
missing_malls

['CITY VIBE',
 'SHAW HOUSE AND CENTRE',
 'CLARKE QUAY CENTRAL',
 'MANDARIN GALLERY',
 'CHANGE ALLEY']

In [264]:
# Manually search for missing malls
def addLocationToList(locationList, mall, lat, long):
    location_details = {}
    location_details['name'] = mall
    location_details['latitude'] = lat
    location_details['longitude'] = long
    locationList.append(location_details)

In [265]:
# Add missing locations
addLocationToList(mall_lat_long, 'CITY VIBE', '1.3142420326981248', '103.76527744903247')
addLocationToList(mall_lat_long, 'SHAW HOUSE AND CENTRE', '1.3063028918746467', '103.83184664246109')
addLocationToList(mall_lat_long, 'CLARKE QUAY CENTRAL', '1.2890266594142623', '103.84660800577113')
addLocationToList(mall_lat_long, 'MANDARIN GALLERY', '1.302089565417662', '103.83634054158641')
addLocationToList(mall_lat_long, 'CHANGE ALLEY', '1.2844197208580772', '103.8518214747385')

In [276]:
# Convert to malls_lat_long to pandas dataframe
df_malls = pd.DataFrame(mall_lat_long)

In [277]:
df_malls

,name,latitude,longitude
0,HARBOURFRONT CENTRE,1.26206336410344,103.818732564507
1,NORTHPOINT CITY,1.42822560922688,103.836419992398
2,SUNTEC CITY MALL,1.29350132535558,103.857307495824
3,CATHAY CINELEISURE ORCHARD,1.30146393496315,103.836463810773
4,HEARTLAND MALL,1.35954177325561,103.885167372726
...,...,...,...
152,CITY VIBE,1.3142420326981248,103.76527744903247
153,SHAW HOUSE AND CENTRE,1.3063028918746467,103.83184664246109
154,MANDARIN GALLERY,1.302089565417662,103.83634054158641
155,CHANGE ALLEY,1.2844197208580772,103.8518214747385
